# Mount the Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Access Gene expression omnibus (GEO) data (GSE84422 with plateform ID GPL96), already downloaded in Google Drive

Access the data from Google Drive downloaded previously



In [ ]:
import pandas as pd
expr_df = pd.read_csv("/content/drive/MyDrive/Gene_Exp_Data.csv", index_col=0)
print("Expression matrix shape:", expr_df.shape)
expr_df.head()

The above file has been uploaded to Google Drive and is ready for AI and ML use. Now we will not run the above codes. We will only remount the drive if needed, add the necessary libraries, define the file path, and use the data as shown below.

# Box plot of GEO Data

Plot few samples only, to reduce the execution time

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Load expression data with probe IDs as index
expr_df = pd.read_csv("/content/drive/MyDrive/Gene_Exp_Data.csv", index_col=0)

# Transpose: rows = samples, columns = probes
df_t = expr_df.transpose()

# Keep only first 20 samples (i.e., first 20 rows)
df_t_subset = df_t.iloc[:20]

# Plot boxplot: one box per sample
plt.figure(figsize=(10, 6))
df_t_subset.transpose().boxplot(
    rot=45,
    grid=False,
    showfliers=False
)
plt.title("Boxplot of First 20 Samples - GSE84422 GPL96")
plt.ylabel("Expression (log2 scale)")
plt.xlabel("Samples")
plt.tight_layout()
plt.show()


The above graph tells that :
The spread and center of expression values across samples look quite similar.

This suggests no major normalization issues or technical outliers in the first 20 samples.

The boxes are nearly aligned, meaning gene expression distribution is comparable across samples which is a a good sign.

Plot for all samples in GEO

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

file_path = '/content/drive/MyDrive/GSE84422_GPL96_expression_data.csv'
# Load expression data with probe IDs as index
expression_data = pd.read_csv(file_path, index_col=0)

# Transpose: rows = samples, columns = probes
df_t = expression_data.transpose()

# Keep only first 20 samples (i.e., first 20 rows)
#df_t_subset = df_t.iloc[:20]

# Plot boxplot: one box per sample
plt.figure(figsize=(500, 6))
df_t.transpose().boxplot(
    rot=45,
    grid=False,
    showfliers=False
)
plt.title("Boxplot of all Samples - GSE84422 GPL96")
plt.ylabel("Expression (log2 scale)")
plt.xlabel("Samples")
plt.tight_layout()
plt.show()

# Expression density plot Gene Exp Data

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(10, 6))
# Transpose so that each sample (column) is plotted
for sample in expr_df.columns:
    sns.kdeplot(expr_df[sample], linewidth=0.5)

plt.title("GSE84422: Expression density")
plt.xlabel("Intensity")
plt.ylabel("Density")
plt.grid(False)
plt.tight_layout()
plt.show()


# Mean variation trend plots

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Load expression data: rows = genes/probes, columns = samples
# Data is already log2-transformed!
# expression_data = pd.read_csv("expression_data.csv", index_col=0)
# Load the expression and metadata CSVs
file_path = '/content/drive/MyDrive/GSE84422_GPL96_expression_data.csv'
expression_data = pd.read_csv(file_path, index_col=0) # Load with ID_REF as index
# Calculate average log-expression (mean) and standard deviation (sigma)
mean_log_expr = expression_data.mean(axis=1)
std_log_expr = expression_data.std(axis=1)

# Plotting
plt.figure(figsize=(8, 6))
plt.scatter(mean_log_expr, std_log_expr, s=1, color='black', alpha=0.6)
plt.title("GSE84422 : Mean-variance trend")
plt.xlabel("Average log-expression")
plt.ylabel("sqrt(sigma)")
plt.text(mean_log_expr.max()-1, std_log_expr.max()-0.1, f"{expression_data.shape[0]} probes", fontsize=10)
plt.grid(False)
plt.show()


# UMAP Plot of Dataset
UMAP plot for entire sample without any grouping

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import umap
from sklearn.preprocessing import StandardScaler

# The'expression_data' is the DataFrame with genes as rows and samples as columns
# and it is already loaded (and optionally filtered)

# Transpose so samples are rows and genes are features
data_t = expr_df.T
data_t.head()
# Scale the data
scaled_data = StandardScaler().fit_transform(data_t)

# UMAP with 15 neighbors
reducer = umap.UMAP(n_neighbors=15, random_state=42)
embedding = reducer.fit_transform(scaled_data)

# Plot
plt.figure(figsize=(6, 6))
plt.scatter(embedding[:, 0], embedding[:, 1], s=30, c='teal')
plt.title("GSE84422/GPL96: UMAP(nbrs=15)")
plt.xlabel('')
plt.ylabel('')
plt.grid(False)
plt.show()

# UMAP Clustering for disease status

UMAP for neuropathological category (seems not a good cluster)

In [ ]:
import pandas as pd
import umap
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

# Load the expression and metadata CSVs
file_path = '/content/drive/MyDrive/GSE84422_GPL96_expression_data.csv'
expr_df = pd.read_csv(file_path, index_col=0) # Load with ID_REF as index

# Load the metadata file saved previously
meta_df = pd.read_csv("/content/drive/MyDrive/geo_accession_and_all_characteristics.csv")

# Transpose expression data so samples are rows and genes are features
expr_df_T = expr_df.T

# Check sample names in both dataframes
print("Expression data samples (first 5):", expr_df.index[:5].tolist())
print("Metadata geo_accession (first 5):", meta_df['geo_accession'][:5].tolist())

# Align metadata with expression data using 'geo_accession'
# Assuming 'geo_accession' column in meta_df contains the sample names (GSM IDs)
meta_df = meta_df.set_index('geo_accession')

# Print indices after setting index
print("Expression data index (first 5):", expr_df_T.index[:5].tolist())
print("Metadata index (first 5):", meta_df.index[:5].tolist())


# Check for samples in expression data not in metadata
missing_samples = expr_df_T.index.difference(meta_df.index)
if len(missing_samples) > 0:
    print(f"Warning: {len(missing_samples)} samples in expression data not found in metadata:", missing_samples.tolist())

# Filter expression data to keep only samples present in metadata
expr_df_T_aligned = expr_df_T.loc[meta_df.index.intersection(expr_df_T.index)]
meta_df_aligned = meta_df.loc[expr_df_T_aligned.index]

# Standardize expression data before UMAP
scaled_data = StandardScaler().fit_transform(expr_df_T_aligned)

# Apply UMAP
reducer = umap.UMAP(random_state=42)
embedding = reducer.fit_transform(scaled_data)

# Extract the 'neuropathological category' for coloring
ad_status = meta_df_aligned['neuropathological category']

# Plot UMAP with colors based on 'neuropathological category'
plt.figure(figsize=(10, 6))
scatter = plt.scatter(embedding[:, 0], embedding[:, 1], c=ad_status.astype('category').cat.codes, cmap='Set1', s=50)
plt.title("GSE84422/GPL96: UMAP Projection Colored by Neuropathological Category")
plt.xlabel("UMAP 1")
plt.ylabel("UMAP 2")

# Add legend
handles, labels = scatter.legend_elements()
legend_labels = ad_status.astype('category').cat.categories
plt.legend(handles, legend_labels, title="Neuropathological Category", bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

# Phenotype (Meta data access)



Phenotype (metadata) data access from previous notebook (GEO84422.IPYNB), 84422,96


 saved file

In [ ]:
import pandas as pd

# Load the metadata file
meta_df.to_csv('/content/drive/MyDrive/geo_accession_and_all_characteristics.csv', index=False)

# Display result
print(" Metadata shape shape:", meta_df.shape)
meta_df.head()

Study of metadata properties

In [ ]:
import pandas as pd
meta_df = pd.read_csv("/content/drive/MyDrive/geo_accession_and_all_characteristics.csv")
meta_df.describe()

# Gene exp data extraction for Volcano plot: differential expression analysis (DEA)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# extract desirable column in a single file
#merged = pd.concat([table_A['x'], table_B['y']], axis=1)
meta_df = pd.read_csv("/content/drive/MyDrive/geo_accession_and_all_characteristics.csv")
# meta_df_T = meta_df.T
meta_df.head(10)
# print(meta_df_T.head(10))
expr_df = pd.read_csv("/content/drive/MyDrive/Gene_Exp_Data.csv", index_col=0)
# expression_data = pd.read_csv("/content/drive/MyDrive/GSE84422_GPL96_expression_data.csv", index_col=0) # Load with ID_REF as index
expr_df_T = expr_df.T
# expr_df_T.head()
# print(expression_data.head())
#data_volc = meta_df.head(10)

Print different disease status

In [ ]:
# Print 9th column (AD status from metadata)
meta_df_T = meta_df.T
Category_AD = meta_df_T.iloc[9]
print(Category_AD)
# Assign new column names
# expression_data.columns = new_headers

In [ ]:
# expression_data_T.index = Category_AD
# expression_data_T.to_csv('/content/drive/MyDrive/GSE84422_GPL96_DEA_data.csv')
# expression_data_T.head()

expr_df_T.index = Category_AD
expr_df_T.to_csv('/content/drive/MyDrive/GSE84422_GPL96_DEA_data.csv')
expr_df_T.head()

In [ ]:
print(expr_df_T.shape)

In [ ]:
DEA_data = pd.read_csv("/content/drive/MyDrive/GSE84422_GPL96_DEA_data.csv")
print("DEA_data", DEA_data.shape)
DEA_data.head()

In [ ]:
DEA_data.tail()

# Extracrting Data for Normal and definite AD status only

Normal vs definite Differential Expression analysis DEA (Volcano plot)

In [ ]:
import pandas as pd


DEA_data = pd.read_csv("/content/drive/MyDrive/GSE84422_GPL96_DEA_data.csv", index_col=0)
# df = pd.read_csv("/content/drive/MyDrive/FILENAME.csv", index_col=0)
# Keep only samples from 'Normal' and 'Definite AD'
DEA_filter_AD = DEA_data[DEA_data.index.isin(['Normal', 'definite AD'])]
DEA_filter_AD.to_csv("/content/drive/MyDrive/DEA_filter_AD.csv")

DEA_filter_AD.head()

In [ ]:
print(DEA_filter_AD.shape)
# now only 542 sample are only Definite of normal

Find transpose of Gene Exp Data

In [ ]:
# This format (transpose) is needed for applying the t-test gene by gene
DEA_filter_AD_T = DEA_filter_AD.T

print(DEA_filter_AD_T)
DEA_filter_AD.head()


# Creating lables for disease status

 Create labels for  two groups
 0 for normal, 1 for definite AD

In [ ]:
import numpy as np

group_labels = DEA_filter_AD.index.values  # ['Normal', 'Normal', 'definite AD', ...]
# print(group_labels)
labels = np.array([1 if g == 'definite AD' else 0 for g in group_labels])
print(labels)

In [ ]:
DEA_filter_AD_T.head()

# Finding p_values and fold change

In [ ]:
from scipy.stats import ttest_ind

p_values = []
logFC = []

for gene in DEA_filter_AD_T.index:
    group1 = DEA_filter_AD_T.loc[gene, labels == 0]
    #expression values of one gene across Normal samples.
    group2 = DEA_filter_AD_T.loc[gene, labels == 1]
 #group2 selects the row(s) for gene from DEA_filter_AD_T and keeps only the columns where labels equals 1.
 # the above code are iteration based, hence storing values as PanadaSeries (not data frame)
 # printing head results in one series

    mean1 = group1.mean()
    mean2 = group2.mean()
#by default computes the mean column-wise (axis=0).


    # logFC.append(np.log2(mean2 + 1e-6) - np.log2(mean1 + 1e-6))  # already log2 transformed
    logFC.append(mean2 - mean1)

    t_stat, p_val = ttest_ind(group1, group2, equal_var=False)
    p_values.append(p_val)


print(mean1.shape)
print(mean2.shape)


Showing the p_values and logFC for each gene

coding for dea_results_new

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

results = []  # to store results for each gene

for gene in DEA_filter_AD_T.index:
    group1 = DEA_filter_AD_T.loc[gene, labels == 0]  # Normal
    group2 = DEA_filter_AD_T.loc[gene, labels == 1]  # Definite AD

    mean1 = group1.mean()
    mean2 = group2.mean()

    log_fc = mean2 - mean1  # already log2 transformed
    t_stat, p_val = ttest_ind(group1, group2, equal_var=False)

    results.append([gene, mean1, mean2, log_fc, p_val])

# Create DataFrame
dea_results_new = pd.DataFrame(results, columns=["Gene", "Mean_Normal", "Mean_DefiniteAD", "log2FC", "p_value"])

# Optional: sort by p-value
# dea_results = dea_results.sort_values(by="p_value")

# View top rows
print(dea_results_new.head())



In [ ]:
print(mean2.shape)
#Saving as DataFrame

# dea_results.to_csv("/content/drive/MyDrive/DEA_Normal_vs_DefiniteAD.csv", index=False)
# print(mean1.shape)
# print(mean2.shape)
DEA_filter_AD_T.loc[:, labels == 0].head()
# group1_T = group1.T
# print(group2.head())
# print(group1.shape)
# print(group2.shape)
# group1.head()

Sorting based on values calculated

In [ ]:
# Sort by p-value for FDR correction

# dea_results_new = dea_results_new.sort_values('p_value').reset_index(drop=True)

# Benjamini-Hochberg adjustment
n = len(dea_results_new)
dea_results_new['adj_pval'] = dea_results_new['p_value'] * n / (np.arange(1, n+1))
dea_results_new['adj_pval'] = dea_results_new['adj_pval'].clip(upper=1)  # p-values can't exceed 1

# Add -log10 p-value for volcano plot
dea_results_new['-log10(pval)'] = -np.log10(dea_results_new['p_value'])

# Save to CSV
dea_results_new.to_csv("/content/drive/MyDrive/DEA_Normal_vs_DefiniteAD_new.csv", index=False)


# View first few rows
print(dea_results_new.head(10))

# Originally, 1007_s_at was just first in the index order (probably from the raw GEO file order).
# After sorting, genes with strongest statistical significance (lowest p_value,or highest FDR) are moved to the top, that is why we now see 214414_x_at instead.
dea_results_new[dea_results_new['Gene'] == '1007_s_at']

Saving the results obtained by

In [ ]:
# import pandas as pd
# dea_results = pd.DataFrame({
#     'Gene': DEA_filter_AD_T.index,
#     'log2FC': logFC,
#     'p_value': p_values
# })

# dea_results.to_csv("/content/drive/MyDrive/DEA_Normal_vs_DefiniteAD.csv", index=False)
# # print(dea_results.head())


# # Adjust p-values using Benjamini-Hochberg (FDR)
# dea_results['adj_pval'] = dea_results['p_value'] * len(dea_results) / (np.arange(1, len(dea_results)+1))
# dea_results = dea_results.sort_values('p_value')
# dea_results['-log10(pval)'] = -np.log10(dea_results['p_value'])
# dea_results.head()

# Volcano plot from the values obtained above

Volcano plot using dea_results_new

In [ ]:
!pip install adjustText

In [ ]:
# import numpy as np
import pandas as pd
dea_results_new = pd.read_csv("/content/drive/MyDrive/DEA_Normal_vs_DefiniteAD_new.csv")
# dea_results_new.head(10)
# Sort by p-value to get top 10 significant genes
dea_results_new.sort_values('p_value').head(10)

In [ ]:

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from adjustText import adjust_text



dea_results_new = pd.read_csv("/content/drive/MyDrive/DEA_Normal_vs_DefiniteAD_new.csv")

# Sort by p-value to get top 10 significant genes
# top10 = dea_results_new.sort_values('p_value').head(10)
# Filter for significance and effect size
filtered = dea_results_new[
    (abs(dea_results_new['log2FC']) > 0.5) &
    (dea_results_new['adj_pval'] < 0.05)
]


print(filtered)

plt.figure(figsize=(10,6))
plt.scatter(dea_results_new['log2FC'],
            dea_results_new['-log10(pval)'],
            color='blue',
            # c=(dea_results_new['adj_pval'] < 0.05),
            cmap='coolwarm',
            alpha=0.7
)
###################
# Highlight DEGs
plt.scatter(filtered['log2FC'], filtered['-log10(pval)'],
            color='red', alpha=0.8, label='|log2FC|>0.5 & FDR<0.05')
##################

plt.axhline(-np.log10(0.05), color='grey', linestyle='--') # -Log10(0.05) = 1.3, p = 0.05)
plt.axvline(0.5, color='green', linestyle='--')
plt.axvline(-0.5, color='green', linestyle='--')

# Label top 10 significant genes

# Use filtered genes instead of top10
texts = []
for _, row in filtered.iterrows():
    texts.append(
        plt.text(
            row['log2FC'],
            row['-log10(pval)'],
            row['Gene'],  # or row['Gene Symbol'] if merged with annotation
            fontsize=8,
            ha='center',
            va='bottom'
        )
    )
# Adjust text to avoid overlap
adjust_text(texts, arrowprops=dict(arrowstyle='-', color='black'))
plt.title("Volcano Plot: Definite AD vs Normal ")
plt.xlabel("log2(Fold Change)")
plt.ylabel("-log10(p-value)")
plt.grid(True)
plt.show()


**Useful Deatils of Volcano plots:**

Axes:
X-axis (log2(Fold Change)):
Measures the magnitude and direction of change.
Left (−): Downregulated genes in Definite AD.
Right (+): Upregulated genes in Definite AD.
Vertical green lines at ±1 show 2-fold change thresholds.
Y-axis (−log10(p-value)):
Measures statistical significance.
The higher the point, the smaller the p-value (i.e., more significant).
The horizontal dashed line typically marks p = 0.05 → −log10(0.05) ≈ 1.3.

**Points:**
Red dots: Genes with p-value < 0.05
Blue dots: Genes with p-value ≥ 0.05
Only the red dots outside the green vertical lines are both:
Statistically significant (p < 0.05), and
Biologically significant (|log2FC| > 1)
These are your most interesting differentially expressed genes (DEGs).

**Points:**
Red dots: Genes with p-value < 0.05
Blue dots: Genes with p-value ≥ 0.05
Only the red dots outside the green vertical lines are both:
Statistically significant (p < 0.05), and
Biologically significant (|log2FC| > 1)
These are the most interesting differentially expressed genes (DEGs).

In [ ]:
dea_results_new.to_csv("/content/drive/MyDrive/DEA_Normal_vs_DefiniteAD_new.csv", index=False)
print("Max log2FC:", dea_results_new["log2FC"].max())
print("Min log2FC:", dea_results_new["log2FC"].min())
print("Max -log10(p-value):", (-np.log10(dea_results_new["p_value"])).max())

Annotation data for Probe ID, Gene Symbol, and others..

In [ ]:
import pandas as pd

# Read uploaded file with some changes
annotation_data = pd.read_csv(
    "/content/drive/MyDrive/GPL96-57554_annot.txt",
    sep='\t',
    comment='#'
)

# Save file with another name (without index column)
annotation_data.to_csv(
    "/content/drive/MyDrive/GSE84422_GPL96_Annotation_Data.csv",
    index=False
)

# If want it later, load it as 'annot':
# annot = pd.read_csv("/content/drive/MyDrive/GSE84422_GPL96_Annotation_Data.csv")

In [ ]:
# Load the saved annotation file
annot = pd.read_csv("/content/drive/MyDrive/GSE84422_GPL96_Annotation_Data.csv")
annot_prob_Gene_ID = annot[["ID", "Gene Symbol", "ENTREZ_GENE_ID"]]
print(annot_prob_Gene_ID.head(20))
# Display the top 20 rows
# print(annot.head(20))
annot[annot['ID'] == '204151_x_at']

In [ ]:
print(annot_prob_Gene_ID.tail(20))

In [ ]:
import pandas as pd
expr_df = pd.read_csv("/content/drive/MyDrive/Gene_Exp_Data.csv", index_col=0)
# expression_data = pd.read_csv("/content/drive/MyDrive/GSE84422_GPL96_expression_data.csv", index_col=0)
print("Expression matrix shape:", expr_df.shape)
expr_df.tail(5)

In [ ]:
import pandas as pd
annot = pd.read_csv("/content/drive/MyDrive/GSE84422_GPL96_Annotation_Data.csv")
# annot_prob_Gene_ID = annot[["ID", "Gene Symbol", "ENTREZ_GENE_ID"]]

dea_results_new = pd.read_csv("/content/drive/MyDrive/DEA_Normal_vs_DefiniteAD_new.csv")
# Merge DEA results with annotation data after mapping
merged = dea_results_new.merge(annot, left_on="Gene", right_on="ID", how="left")

# Keep only genes that meet the thresholds
filtered_merged = merged[
    (abs(merged['log2FC']) > 0.5) &
    (merged['adj_pval'] < 0.05)
]
# "ENTREZ_GENE_ID"
# Print the filtered merged data
print(filtered_merged[['Gene', 'Gene Symbol','ENTREZ_GENE_ID', 'log2FC', 'p_value', 'adj_pval']])


Remove only duplicate genes AND KEEP IMP DATA

In [ ]:
import pandas as pd
annot = pd.read_csv("/content/drive/MyDrive/GSE84422_GPL96_Annotation_Data.csv")
# annot_prob_Gene_ID = annot[["ID", "Gene Symbol", "ENTREZ_GENE_ID"]]

dea_results_new = pd.read_csv("/content/drive/MyDrive/DEA_Normal_vs_DefiniteAD_new.csv")
# Merge DEA results with annotation data after mapping
merged = dea_results_new.merge(annot, left_on="Gene", right_on="ID", how="left")

# Keep only genes that meet the thresholds
filtered_merged = merged[
    (abs(merged['log2FC']) > 0.5) &
    (merged['adj_pval'] < 0.05)
]
# "ENTREZ_GENE_ID"
# Print the filtered merged data
# print(filtered_merged)
# print(filtered_merged[['Gene Symbol','ENTREZ_GENE_ID', 'log2FC', 'p_value', 'adj_pval']])

# Remove duplicate genes based on 'Gene Symbol'
filtered_unique = filtered_merged.drop_duplicates(subset=['Gene Symbol'], keep='first')

# Display the cleaned dataframe
filtered_unique_all_data = filtered_unique[['Gene Symbol', 'ENTREZ_GENE_ID', 'log2FC', 'p_value', 'adj_pval']]
print(filtered_unique_all_data)
#sig_genes_df.to_csv('/content/drive/MyDrive/signi_genes.csv', index=False)
filtered_unique_all_data.to_csv('/content/drive/MyDrive/filtered_unique_all_data.csv', index=False)
# sig_genes_all_info = filtered_merged[['Gene Symbol','ENTREZ_GENE_ID', 'log2FC', 'p_value', 'adj_pval']].dropna().unique().tolist()
# print(sig_genes_all_info)

List of signifcant genes obtained from DEA

In [ ]:
import pandas as pd
# Use Gene Symbol column, drop duplicates & NaN
sig_genes = filtered_merged['Gene Symbol'].dropna().unique().tolist()

# Convert the list to a DataFrame
sig_genes_df = pd.DataFrame(sig_genes, columns=["Gene Volcano"])

# Save to CSV in drive
sig_genes_df.to_csv('/content/drive/MyDrive/signi_genes.csv', index=False)
# meta_df.to_csv('/content/drive/MyDrive/geo_accession_and_all_characteristics.csv', index=False)
print("Significant genes saved to drive as 'signi_genes'.csv'")

print(f"Number of significant genes: {len(sig_genes)}")
print(sig_genes)  # Preview first 10


# UMAP plot for Normal and definite AD only

In [ ]:
import pandas as pd
import umap
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

# Load expression and metadata
expr_df = pd.read_csv('/content/drive/MyDrive/GSE84422_GPL96_expression_data.csv', index_col=0)
meta_df = pd.read_csv("/content/drive/MyDrive/geo_accession_and_all_characteristics.csv")

# Transpose expression data (samples as rows)
expr_df_T = expr_df.T

# Set geo_accession as index
meta_df = meta_df.set_index('geo_accession')

# Align expression with metadata
expr_df_T_aligned = expr_df_T.loc[expr_df_T.index.intersection(meta_df.index)]
meta_df_aligned = meta_df.loc[expr_df_T_aligned.index]

# Filter only Normal and Definite AD samples
mask = meta_df_aligned['neuropathological category'].isin(['Normal', 'Definite AD'])
expr_df_filtered = expr_df_T_aligned[mask]
meta_df_filtered = meta_df_aligned[mask]

# Highlighting expression of 17 significant genes

# List of 17 significant genes (probe IDs from volcano plot, replace with actual if different)
sig_genes = [
    "202912_at", "202917_s_at", "203535_at", "203915_at", "204018_x_at", "204712_at",
    "205048_s_at", "205230_at", "207574_s_at", "208151_x_at", "208719_s_at", "209047_at",
    "209116_x_at", "209183_s_at", "210512_s_at", "211696_x_at", "213479_at"
]

# Check which of these genes are in the expression data
available_genes = [g for g in sig_genes if g in expr_df_filtered.columns]
missing_genes = list(set(sig_genes) - set(available_genes))
print(f"Missing genes: {missing_genes}")

# Calculate average expression per sample for significant genes
sig_expr = expr_df_filtered[available_genes]
avg_sig_expr = sig_expr.mean(axis=1)

# Scale expression data
scaled_data = StandardScaler().fit_transform(expr_df_filtered)

# Apply UMAP
reducer = umap.UMAP(random_state=42)
embedding = reducer.fit_transform(scaled_data)

# Plot with color representing average expression of significant genes
plt.figure(figsize=(10, 6))
scatter = plt.scatter(embedding[:, 0], embedding[:, 1], c=avg_sig_expr, cmap='viridis', s=50)
plt.title("UMAP (Normal vs Definite AD) - Colored by Avg Expression of 17 Significant Genes")
plt.xlabel("UMAP 1")
plt.ylabel("UMAP 2")
cbar = plt.colorbar(scatter)
cbar.set_label("Avg Expression (17 genes)")
plt.tight_layout()
plt.show()


In [ ]:
# ACCESS SIGNIFICANT GENES
sig_genes = pd.read_csv("/content/drive/MyDrive/signi_genes.csv")
sig_genes_df = pd.DataFrame(sig_genes, columns=["Gene Volcano"])
# print(sig_genes)
print(sig_genes_df)

# Random forest analysis
To find rank list of the genes using random forest, and then finding intersection of genes with Volcano Genes

In [ ]:
# DEA_data = pd.read_csv("/content/drive/MyDrive/GSE84422_GPL96_DEA_data.csv", index_col=0)
# # df = pd.read_csv("/content/drive/MyDrive/FILENAME.csv", index_col=0)
# # Keep only samples from 'Normal' and 'Definite AD'
# DEA_filter_AD = DEA_data[DEA_data.index.isin(['Normal', 'definite AD'])]
# DEA_filter_AD.to_csv("/content/drive/MyDrive/DEA_filter_AD.csv")

# DEA_filter_AD.head()


import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import cross_val_score
# -------------------------------
# 1. Load data
# -------------------------------
data = pd.read_csv("/content/drive/MyDrive/DEA_filter_AD.csv", index_col=0)

# Separate labels (first column) and features (all gene columns)
y = data.index   # 'Normal' or 'definite AD'
X1 = data.values  # all gene expression values #unfiltered

##############################################################

gene_names = data.columns

print("Initial dataset size:", X1.shape)  # (samples, genes)

# -------------------------------
# 2. Filter low-variance genes
# -------------------------------
# This removes genes with very low variance
selector = VarianceThreshold(threshold=0.0)  # adjust threshold if needed
X = selector.fit_transform(X1)
filtered_gene_names = gene_names[selector.get_support()]

print("Dataset size after filtering:", X.shape)

# -------------------------------
# 3. Train-test split
# -------------------------------
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)
#############################################################
# -------------------------------
# 2. Train-test split
# -------------------------------
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)
# 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 252
# -------------------------------
# 3. Train Random Forest
# -------------------------------
# rf = RandomForestClassifier(
#     n_estimators=100,
#     random_state=42,
#     n_jobs=-1,
#     class_weight="balanced"
# )
rf = RandomForestClassifier(
    n_estimators=252,
    random_state=42,
    min_samples_leaf = 4,
    min_samples_split = 10,
    n_jobs=-1,
    class_weight="balanced"
)

rf.fit(X_train, y_train)

# -------------------------------
# 4. Evaluate
# -------------------------------
y_pred = rf.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))

# -------------------------------
# 5. Feature importance
# -------------------------------
importances = rf.feature_importances_
gene_names = data.columns  # gene IDs from columns

gene_importances = pd.DataFrame({
    "Gene": gene_names,
    "Importance": importances
}).sort_values(by="Importance", ascending=False)

print(gene_importances)
# 6. Load annotation and merge
# -------------------------------
annot = pd.read_csv("/content/drive/MyDrive/GSE84422_GPL96_Annotation_Data.csv")

# Keep only relevant columns
annot = annot[["ID", "Gene Symbol", "ENTREZ_GENE_ID"]]
# So after this line, annot is a slimmed-down dataframe with just these three columns.
# The merge is done with:
# left_on="Gene" → use the "Gene" column from gene_importances
# right_on="ID" → match with "ID" column from annot
# how="left" → keep all rows from gene_importances even if some genes don’t have a matching annotation. For those without a match, the "Gene Symbol" and "ENTREZ_GENE_ID" will be NaN.
importance_annot = gene_importances.merge(
    annot,
    left_on="Gene",   # from gene_importances
    right_on="ID",    # from annotation
    how="left"
)

# Save results
importance_annot.to_csv("/content/drive/MyDrive/gene_ranking_randomforest.csv", index=False)

print("Top 50 important genes:")
print(importance_annot.head(50))


Hyper parameter tuning to refine the results from Random Forest

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import VarianceThreshold
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.metrics import accuracy_score
from scipy.stats import randint

data = pd.read_csv("/content/drive/MyDrive/DEA_filter_AD.csv", index_col=0)

# Separate labels (first column) and features (all gene columns)
y = data.index   # 'Normal' or 'definite AD'
X = data.values  # all gene expression values #unfiltered
# -------------------------------
# 1. Split Data
# -------------------------------
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)
# -------------------------------
# 2. Define Hyperparameter Space
# -------------------------------
param_dist = {
    'n_estimators': randint(50, 300),
    # 'max_depth': randint(1, 5)  # 'None' is not added
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4],
    # "max_features": ["sqrt", "log2", None],
    # "bootstrap": [True, False]


}

# -------------------------------
# 3. Randomized Search
# -------------------------------
rf = RandomForestClassifier(random_state=42, class_weight="balanced", n_jobs=-1)

random_search = RandomizedSearchCV(
    estimator=rf,
    param_distributions=param_dist,
    n_iter=5,              # try 10 random combinations
    cv=5,                   # 5-fold cross validation
    scoring="f1_macro",     # metric can be changed
    verbose=2,
    random_state=42,
    n_jobs=-1
)

random_search.fit(X_train, y_train)

print("Best Parameters:", random_search.best_params_)

# -------------------------------
# 4. Evaluate Best Model
# -------------------------------
best_rf = random_search.best_estimator_
y_pred = best_rf.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
# Best Parameters: {'max_depth': 3, 'n_estimators': 229}

##############################################################
# Fitting 5 folds for each of 5 candidates, totalling 25 fits
# Best Parameters: {'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 252}
# Accuracy: 0.8440366972477065
############################################################

Cross validation 4 fold

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
data = pd.read_csv("/content/drive/MyDrive/DEA_filter_AD.csv", index_col=0)

# Separate labels (first column) and features (all gene columns)
y = data.index   # 'Normal' or 'definite AD'
X = data.values  # all gene expression values #unfiltered
 # Cross validation
rf = RandomForestClassifier(n_estimators=252, random_state=42, min_samples_leaf = 4, min_samples_split = 10) #'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 252
scores = cross_val_score(rf, X, y, cv=5)  # 5-fold CV
print("Mean accuracy:", scores.mean())


# Plotting confusion matrix and confusion heatmap plots using the results of Random Forest

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
# Assuming y_test are true labels and y_pred are predictions
print("Classification Report:\n")
print(classification_report(y_test, y_pred))

print("\nConfusion Matrix:\n")
print(confusion_matrix(y_test, y_pred))

cm = confusion_matrix(y_test, y_pred)

plt.figure(figsize=(6,4))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=["Normal","definite AD"], yticklabels=["Normal","definite AD"])
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Confusion Matrix")
plt.show()

DEG list

In [ ]:
# sig_genes_df.to_csv('/content/drive/MyDrive/signi_genes.csv', index=False)

SIGN_gene = pd.read_csv("/content/drive/MyDrive/signi_genes.csv")
print(SIGN_gene)

# Genes intersection
From volcano genes to Random forest high ranked genes (Top 100)

We got 10 common genes from DEG and RF

In [ ]:
import pandas as pd

SIGN_gene = pd.read_csv("/content/drive/MyDrive/signi_genes.csv")
print(SIGN_gene)
# SIGN_gene = pd.read_csv("/content/drive/MyDrive/signi_genes.csv")
importance_annot = pd.read_csv("/content/drive/MyDrive/gene_ranking_randomforest.csv")
importance_annot_top = importance_annot.head(int(100)) # for top 1% write int(22283 * 0.01)
# Get unique genes from each dataframe
genes1 = set(SIGN_gene['Gene Volcano'].dropna().unique())
genes2 = set(importance_annot_top['Gene Symbol'].dropna().unique())

# Find common genes
common_genes = sorted(genes1.intersection(genes2))

print(f"Number of common genes: {len(common_genes)}")
print(common_genes[:20])  # Preview first 20

# list to datafram
common_genes_df = pd.DataFrame(common_genes, columns=["DEG_and_RF"])
# Save to CSV
common_genes_df.to_csv("/content/drive/MyDrive/DEG_and_RF.csv", index=False)
print(common_genes_df)

# Support Vector Machine Analysis
**To filter out more reliable gene biomarkers**

Using SVM as a disease condition predictor

In [ ]:
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler

############################################

# DEA_data = pd.read_csv("/content/drive/MyDrive/GSE84422_GPL96_DEA_data.csv", index_col=0)
# df = pd.read_csv("/content/drive/MyDrive/FILENAME.csv", index_col=0)
# Keep only samples from 'Normal' and 'Definite AD'
# DEA_filter_AD = DEA_data[DEA_data.index.isin(['Normal', 'definite AD'])]
DEA_filter_AD = pd.read_csv("/content/drive/MyDrive/DEA_filter_AD.csv",index_col=0)

DEA_filter_AD.head()

###########################################

# Assuming we already have DEA_filter_AD
X = DEA_filter_AD.values          # expression matrix
y = DEA_filter_AD.index.values    # labels: 'Normal' or 'definite AD'

# gene_names = DEA_filter_AD.columns


In [ ]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Standardize (important for SVM)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Train Linear SVM
# C (Regularization Parameter)
# Controls how much SVM tolerates misclassification.
# High C (e.g., 100) → Model tries to classify every point correctly → tighter boundaries, risk of overfitting.
# Low C (e.g., 0.01) → Allows some misclassifications → smoother boundary, better generalization.
clf = SVC(kernel="linear", C=50, class_weight="balanced", random_state=42)
# clf = SVC(kernel="linear", C=1, probability=True, random_state=42)
clf.fit(X_train, y_train)

print("Test Accuracy:", clf.score(X_test, y_test))
# Test Accuracy: 0.7431192660550459

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, cross_val_score

scores = cross_val_score(clf, scaler.fit_transform(X), y, cv=5)
print("Mean CV Accuracy:", scores.mean())

Confusion matrix

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score

y_pred = clf.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

cm = confusion_matrix(y_test, y_pred, labels=clf.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=clf.classes_)
disp.plot(cmap="Blues")
plt.title("Confusion Matrix (SVM)")
plt.show()

# Genes from SVM

Ranking genes on the basis of their weights

In [ ]:
# Extract weights from linear SVM
coef = np.ravel(clf.coef_)

# Gene rankings
gene_importance_SVM = pd.DataFrame({
    "Gene": gene_names,
    "Weight": coef,
    "AbsWeight": np.abs(coef)
}).sort_values(by="AbsWeight", ascending=False)

# print(gene_importance_SVM)
##############################################################
# Annotation data
annot = pd.read_csv("/content/drive/MyDrive/GSE84422_GPL96_Annotation_Data.csv")
# print(annot)
# Keep only relevant columns
annot = annot[["ID", "Gene Symbol", "ENTREZ_GENE_ID"]]
# So after this line, annot is a slimmed-down dataframe with just these three columns.

# importance_annot = gene_importances.merge(
#     annot,
#     left_on="Gene",   # from gene_importances
#     right_on="ID",    # from annotation
#     how="left"
# )
# The merge is done with:
# left_on="Gene" → use the "Gene" column from gene_importances
# right_on="ID" → match with "ID" column from annot
# how="left" → keep all rows from gene_importances even if some genes don’t have a matching annotation. For those without a match, the "Gene Symbol" and "ENTREZ_GENE_ID" will be NaN.
importance_annot_SVM = gene_importance_SVM.merge(
    annot,
    left_on="Gene",   # from gene_importances
    right_on="ID",    # from annotation
    how="left"
)

# Save also
importance_annot_SVM.to_csv("/content/drive/MyDrive/gene_importance_SVM.csv", index=False)

print("Top 50 important genes:")
print(importance_annot_SVM.head(50))
############################################################


# **Gene intersection from DEG and SVM**
Intersection of high ranked genes from SVM (Top 100)
We got 7 intersected genes (now 8)

['ADM', 'C10orf10', 'DDX17', 'HBA1 /// HBA2', 'HBB', 'PSPH', 'VEGFA']

In [ ]:
# Top 100 genes
import pandas as pd
SIGN_gene = pd.read_csv("/content/drive/MyDrive/signi_genes.csv")
# top_SVM_genes = gene_importance_SVM.head(10)

annot = pd.read_csv("/content/drive/MyDrive/GSE84422_GPL96_Annotation_Data.csv")
# Keep only relevant columns
annot = annot[["ID", "Gene Symbol", "ENTREZ_GENE_ID"]]
#############################################################

#The merge is done with:
# left_on="Gene" → use the "Gene" column from gene_importances
# right_on="ID" → match with "ID" column from annot
# how="left" → keep all rows from gene_importances even if some genes don’t have a matching annotation.
# For those without a match, the "Gene Symbol" and "ENTREZ_GENE_ID" will be NaN.

# SIGN_gene = pd.read_csv("/content/drive/MyDrive/signi_genes.csv")
gene_importance_SVM = pd.read_csv("/content/drive/MyDrive/gene_importance_SVM.csv")
gene_importance_SVM_top = gene_importance_SVM.head(int(100)) # for top 1% write int(22283 * 0.01)
# Get unique genes from each dataframe

genes11 = set(SIGN_gene['Gene Volcano'].dropna().unique())
genes22 = set(gene_importance_SVM_top['Gene Symbol'].dropna().unique())
# print(genes11)
# print(genes22)
# Find common genes
common_genes_SVM = sorted(genes11.intersection(genes22))
# list to datafram
common_genes_SVM_df = pd.DataFrame(common_genes_SVM, columns=["DEG_and_SVM"])
# Save to CSV
common_genes_SVM_df.to_csv("/content/drive/MyDrive/DEG_and_SVM.csv", index=False)
# common_genes
print(f"Number of common genes: {len(common_genes_SVM)}")
print(common_genes_SVM_df)

# **SVM-RFE**
**(Another method to find high ranked genes)**

Previus SVM could not be improved more!
SVM-RFE (Recursive Feature Elimination), the workflow would be different:
Start with all genes (or after filtering by variance, if dataset is too large).
Train an SVM (usually linear kernel for interpretability).
Rank features by their weights (importance).
Recursively eliminate the least important features.
Repeat until the desired number of genes is selected.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn.feature_selection import RFE, VarianceThreshold

# --------------------------
# 1. Load Data
# --------------------------
DEA_filter_AD = pd.read_csv("/content/drive/MyDrive/DEA_filter_AD.csv", index_col=0)

# Extract X (features) and y (labels)
X1 = DEA_filter_AD.values           # expression matrix
y = DEA_filter_AD.index.values      # labels: 'Normal' or 'definite AD'
gene_names = DEA_filter_AD.columns  # original gene IDs

print("Original shape:", X1.shape)

# --------------------------
# 2. Filter by variance
# --------------------------
selector = VarianceThreshold(threshold=0.01)
X = selector.fit_transform(X1)

# Update gene names after filtering
selected_genes = gene_names[selector.get_support()]

print("After variance filtering:", X.shape)

# --------------------------
# 3. Run Recursive Feature Elimination (SVM-RFE)
# --------------------------
svm = SVC(kernel="linear", C=1, random_state=42)
rfe = RFE(estimator=svm, n_features_to_select=100, step=0.1)  # Select top 20 genes
rfe.fit(X, y)

# --------------------------
# 4. Rank features
# --------------------------
ranking = pd.DataFrame({
    "Gene": selected_genes,
    "Rank": rfe.ranking_
}).sort_values(by="Rank")

print("Top 20 genes from SVM-RFE:")
print(ranking.head(100))



In [ ]:
annot = pd.read_csv("/content/drive/MyDrive/GSE84422_GPL96_Annotation_Data.csv")
# print(annot)
# Keep only relevant columns
annot = annot[["ID", "Gene Symbol", "ENTREZ_GENE_ID"]]
# So after this line, annot is a slimmed-down dataframe with just these three columns.

# importance_annot = gene_importances.merge(
#     annot,
#     left_on="Gene",   # from gene_importances
#     right_on="ID",    # from annotation
#     how="left"
# )
# The merge is done with:
# left_on="Gene" → use the "Gene" column from gene_importances
# right_on="ID" → match with "ID" column from annot
# how="left" → keep all rows from gene_importances even if some genes don’t have a matching annotation.
# For those without a match, the "Gene Symbol" and "ENTREZ_GENE_ID" will be NaN.
importance_annot_SVM_RFE = ranking.merge(
    annot,
    left_on="Gene",   # from gene_importances
    right_on="ID",    # from annotation
    how="left"
)

# Save also
importance_annot_SVM_RFE.to_csv("/content/drive/MyDrive/gene_importance_SVM.csv", index=False)

print("Top 20 important genes:")
print(importance_annot_SVM_RFE.head(50))

# Gene intersection from DEG and SVM-RFE

Here we got 8 intersected genes

In [ ]:
# Top 20 genes
import pandas as pd
SIGN_gene = pd.read_csv("/content/drive/MyDrive/signi_genes.csv")
# top_SVM_genes = gene_importance_SVM.head(10)
print()
annot = pd.read_csv("/content/drive/MyDrive/GSE84422_GPL96_Annotation_Data.csv")
# Keep only relevant columns
annot = annot[["ID", "Gene Symbol", "ENTREZ_GENE_ID"]]
#############################################################
importance_annot_SVM_RFE = pd.read_csv("/content/drive/MyDrive/gene_importance_SVM.csv")

# gene_importance_SVM = pd.read_csv("/content/drive/MyDrive/gene_importance_SVM.csv")
# gene_importance_SVM_top = gene_importance_SVM.head(int(100)) # for top 1% write int(22283 * 0.01)
# Get unique genes from each dataframe

genes11 = set(SIGN_gene['Gene Volcano'].dropna().unique())
genes22 = set(importance_annot_SVM_RFE.head(100)['Gene Symbol'].dropna().unique())
# print(genes11)
# print(genes22)
# Find common genes
common_genes_SVM_RFE = sorted(genes11.intersection(genes22))
# common_genes
print(f"Number of common genes: {len(common_genes_SVM_RFE)}")
# list to dataframe
common_genes_SVM_RFE_df = pd.DataFrame(common_genes_SVM, columns=["DEG_and_SVMRFE"])
# Save to CSV
common_genes_SVM_RFE_df.to_csv("/content/drive/MyDrive/DEG_and_SVMRFE.csv", index=False)
print(common_genes_SVM_RFE)

In [ ]:
!ls

In [ ]:
import os

# Updated path, assuming the notebook is inside 'Colab Notebooks' folder
notebook_path = "/content/drive/MyDrive/Colab Notebooks/GSE84422_Biomarker_Genes_Strip.ipynb"
output_path = "/content/drive/MyDrive/abc_no_code.ipynb"

# Ensure the notebook file exists before proceeding
if not os.path.exists(notebook_path):
    print(f"Error: Notebook not found at {notebook_path}. Please verify the filename and path.")
else:
    !jupyter nbconvert --to notebook --ClearOutputPreprocessor.enabled=True \
    --TemplateExporter.exclude_input=True \
    --output="{output_path}" "{notebook_path}"


In [ ]:
import os
print(os.getcwd())